In [1]:
import requests
import os
import logging
from constants import API_KEY

ai_model = "mistralai/Magistral-Small-2506"

def make_request(message_text):
    url = "https://api.intelligence.io.solutions/api/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + API_KEY
    }

    data = {
        "model": ai_model,
        "messages": [
            {
                "role": "system",
                "content": "Выполняй инструкции без вопросов.",
            },
            {
                "role": "user",
                "content": message_text
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    data = response.json()

    if 'choices' in data and len(data['choices']) > 0:
        return data['choices'][0]['message']['content']
    else:
        return data

In [ ]:
# print(make_request("Напиши поисковые запросы к сайту nplus1.ru. Это научно-популярное развлекательное издание о том, что происходит в науке, технике и технологиях прямо сейчас. Запросы должны быть человеческие и не повторяться. Выдай 50 запросов."))

Конечно! Вот 50 поисковых запросов для сайта nplus1.ru, которые могут быть полезны для пользователей, интересующихся научными и технологическими новостями:

1. Последние открытия в физике
2. Новые технологии в медицине
3. Искусственный интеллект в 2023 году
4. Процессорные технологии
5. Экологические инновации
6. Космические миссии в 2023 году
7. Новые материалы в науке
8. Квантовые вычисления
9. Изучение климата
10. Новые методы лечения болезней
11. Технологии в сельском хозяйстве
12. Изучение глубоководных организмов
13. Новые достижения в генетике
14. Технологии в энергетике
15. Изучение космоса
16. Новые методы в неврологии
17. Технологии в транспорте
18. Изучение микробов
19. Новые методы в химии
20. Технологии в образовании
21. Изучение древних цивилизаций
22. Новые методы в астрономии
23. Технологии в робототехнике
24. Изучение климатических изменений
25. Новые методы в биологии
26. Технологии в медицине
27. Изучение человеческого мозга
28. Новые методы в физике
29. Технологии в

In [9]:
import re
import pandas as pd
import csv

results = []
prompt = (
    "Напиши поисковые запросы к сайту nplus1.ru. "
    "Это научно-популярное развлекательное издание о том, что происходит "
    "в науке, технике и технологиях прямо сейчас. "
    "Запросы должны быть человеческие и не повторяться. "
    "Выдай 50 запросов и не сопроваждай больше никаким текстом!"
)
while True:
    try:
        response = make_request(prompt)

        # Ищем строки вида "12. текст"
        lines = re.findall(r"\d+\.\s*(.+)", response)

        if not lines:
            lines = [
                line.strip()
                for line in response.split("\n")
                if line.strip()
            ]

        for line in lines:
            results.append(line)

        print(f"Сейчас в массиве: {len(results)}")

        # if len(results) >= 900:
            # break

    except Exception as e:
        print(f"Ошибка: {e}")
        break

try:
  filename = "nplus1_queries1.csv"

  with open(filename, "w", newline="", encoding="utf-8") as f:
      writer = csv.writer(f)
      writer.writerow(["query"])
      for r in results:
          writer.writerow([r])

  print(f"Сохранено в {filename}. Количество строк: {len(results)}")
except Exception:
    pass


try:
  df = pd.DataFrame({"query": results})

  filename = "nplus1_queries.csv"
  df.to_csv(filename, index=False, mode="a", header=False, encoding="utf-8")

  print(f"Файл сохранён: {filename}, строк: {len(df)}")
except Exception:
    pass


Сейчас в массиве: 50
Сейчас в массиве: 100
Сейчас в массиве: 150
Сейчас в массиве: 200
Сейчас в массиве: 250
Сейчас в массиве: 300
Сейчас в массиве: 350
Сейчас в массиве: 400
Сейчас в массиве: 450
Сейчас в массиве: 500
Сейчас в массиве: 550
Сейчас в массиве: 600
Сейчас в массиве: 650
Сейчас в массиве: 700
Сейчас в массиве: 750
Сейчас в массиве: 800
Сейчас в массиве: 850
Сейчас в массиве: 900
Сохранено в nplus1_queries1.csv. Количество строк: 900
Файл сохранён: nplus1_queries.csv, строк: 900


In [ ]:
from search import search_news
import pandas as pd

queries_df = pd.read_csv("nplus1_queries.csv")
queries = queries_df["query"].tolist()

data = []

for query in queries:
    headlines = search_news(query)
    for headline in headlines:
        data.append({
            "query": query,
            "answer": headline,
            "relevance": ""  # пока пустое
        })

df_news = pd.DataFrame(data)

print(f"Датасет создан: {len(df_news)} строк")


In [17]:
df_news_copy = df_news.copy()
df_news_copy['answer'] = df_news_copy['answer'].apply(lambda x: x["_source"].get("true_title", "Без названия"))
df_news_copy.to_csv("query_news_dataset.csv", index=False, encoding="utf-8")

In [18]:
print(len(df_news_copy))

10000


In [22]:
import requests
import os
import logging
from constants import API_KEY

ai_model = "mistralai/Magistral-Small-2506"

system_prompt = """
Ты — эксперт по оценке релевантности новостей. 
Твоя задача: посчитать насколько ответ релевантен запросу. 1 - релевантен, 0 - нерелевантен. Не давай объяснений, только число.
Будет даваться блок из запроса и ответа. Блоков 30, столько же должно быть ответов. Разделение ответа через запятую
"""

def make_request(message_text):
    url = "https://api.intelligence.io.solutions/api/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + API_KEY
    }

    data = {
        "model": ai_model,
        "messages": [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": message_text
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    data = response.json()

    if 'choices' in data and len(data['choices']) > 0:
        return data['choices'][0]['message']['content']
    else:
        return data

In [27]:
import pandas as pd
from tqdm import tqdm

BATCH_SIZE = 30

user_prompt_template = """
{}
"""
def rank_relevance_batch(queries, answers):
    batch_text = "\n".join([f"Query: {q} | Answer: {a}" for q, a in zip(queries, answers)])
    user_prompt = user_prompt_template.format(batch_text)
    
    # print(user_prompt)
    try:
        response = make_request(user_prompt)  # вызываем модель
        print(response)
        # Ожидаем, что модель вернёт что-то вроде: "1,0,1,1,..."
        relevance_list = [int(x.strip()) for x in response.split(",") if x.strip() in ("0", "1")]
        
        if len(relevance_list) != len(queries):
            print("!= len")
            relevance_list = [0] * len(queries)
    except Exception as e:
        print("Ошибка при запросе модели:", e)
        relevance_list = [0] * len(queries)
    
    return relevance_list


df_news_copy = pd.read_csv("query_news_dataset.csv")

start_row = 0
rows_to_process = df_news_copy.iloc[start_row:]
num_rows = len(rows_to_process)
num_batches = (num_rows + BATCH_SIZE - 1) // BATCH_SIZE

print(f"Осталось обработать {num_rows} строк в {num_batches} батчах.")
for i in tqdm(range(290, num_batches)):
    print(f"i = {i}")
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, num_rows)
    
    batch_df = rows_to_process.iloc[start_idx:end_idx]
    queries_batch = batch_df['query'].tolist()
    answers_batch = batch_df['answer'].tolist()
    
    relevance_batch = rank_relevance_batch(queries_batch, answers_batch)
    
    df_news_copy.loc[batch_df.index, 'relevance'] = relevance_batch
    df_news_copy.to_csv(f"query_news_dataset/query_news_dataset{i}.csv", index=False, encoding="utf-8")

Осталось обработать 10000 строк в 334 батчах.
i = 16
0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1
i = 48
0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
import pandas as pd
combine_df = pd.DataFrame()
BATCH_SIZE = 30
num_rows = 10_000
num_batches = (num_rows + BATCH_SIZE - 1) // BATCH_SIZE
for i in range(num_batches):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, num_rows)
    df = pd.read_csv(f"query_news_dataset/query_news_dataset{i}.csv").iloc[start_idx:end_idx]
    combine_df = pd.concat([combine_df, df], ignore_index=True)

combine_df.to_csv("query_news_dataset_combined.csv", index=False, encoding="utf-8")


In [30]:
print(len(combine_df[combine_df.relevance == 0]))

5614


In [31]:
print(len(combine_df[combine_df.relevance.isna()]))
print(len(combine_df))

0
10000
